In [191]:
import pandas as pd
df = pd.read_csv('Data/xerces-1.2.csv')
df = df.drop(['name', 'version', 'name.1'], axis=1)

In [192]:
import numpy as np

X_sample = np.array(df.drop(['bug'], axis=1), dtype='float32')
y_sample = np.array(df['bug'], dtype='uint8')

y_sample[y_sample>0] = 1

print(y_sample.shape)

(440,)


In [193]:
df = pd.read_csv('Data/tomcat.csv')
df = df.drop(['name', 'version', 'name.1'], axis=1)

In [194]:
X_target = np.array(df.drop(['bug'], axis=1), dtype='float32')
y_target = np.array(df['bug'], dtype='uint8')

y_target[y_target>0] = 1

In [195]:
from sklearn.model_selection import train_test_split

X_sample_train, X_sample_test, y_sample_train, y_sample_test = train_test_split(X_sample, y_sample, test_size=.3, random_state=43)

In [196]:
from sklearn.ensemble import BaggingClassifier
from sklearn.metrics import make_scorer, f1_score, matthews_corrcoef
from sklearn.naive_bayes import GaussianNB
from sklearn.model_selection import RandomizedSearchCV
from skopt import BayesSearchCV

params_NB = {'var_smoothing': np.logspace(0,-9, num=100)}
param_grid = {
 'base_estimator' : [GaussianNB()],
 'bootstrap': [True, False],
 'bootstrap_features': [True, False],    
 'n_estimators': [5, 10, 15],
 'max_samples' : [0.4, 0.6, 0.8, 1.0],
 'max_features' : [0.4, 0.6, 0.8, 1.0],
 'base_estimator__var_smoothing' : np.logspace(0,-9, num=500)
}
clf = RandomizedSearchCV(estimator=BaggingClassifier(), 
                    param_distributions=param_grid,
                 n_iter=50,
                 scoring=make_scorer(f1_score),
                 cv=5)

clf.fit(X_sample_train, y_sample_train)

RandomizedSearchCV(cv=5, estimator=BaggingClassifier(), n_iter=50,
                   param_distributions={'base_estimator': [GaussianNB(var_smoothing=1.838776980082327e-06)],
                                        'base_estimator__var_smoothing': array([1.00000000e+00, 9.59320948e-01, 9.20296681e-01, 8.82859884e-01,
       8.46945981e-01, 8.12493021e-01, 7.79441575e-01, 7.47734631e-01,
       7.17317494e-01, 6.88137699e-01, 6.6014490...
       1.57905331e-09, 1.51481892e-09, 1.45319752e-09, 1.39408283e-09,
       1.33737286e-09, 1.28296980e-09, 1.23077980e-09, 1.18071285e-09,
       1.13268257e-09, 1.08660611e-09, 1.04240401e-09, 1.00000000e-09]),
                                        'bootstrap': [True, False],
                                        'bootstrap_features': [True, False],
                                        'max_features': [0.4, 0.6, 0.8, 1.0],
                                        'max_samples': [0.4, 0.6, 0.8, 1.0],
                                        'n

In [197]:
from sklearn.metrics import f1_score, matthews_corrcoef, accuracy_score

y_sample_pred = clf.predict(X_sample_test)

print(f1_score(y_sample_test, y_sample_pred))
print(matthews_corrcoef(y_sample_test, y_sample_pred))

0.2
0.11856381003147767


In [198]:
y_target_pred = clf.predict(X_target)

print(f1_score(y_target, y_target_pred))
print(matthews_corrcoef(y_target, y_target_pred))

0.4154589371980676
0.3563572849201563


In [199]:
from sklearn.metrics import confusion_matrix, recall_score

def GMean_and_Balance(y_true, y_pred):
    CM = confusion_matrix(y_true, y_pred)
    TN = CM[0][0]
    FN = CM[1][0]
    TP = CM[1][1]
    FP = CM[0][1]
    
    recall = recall_score(y_true, y_pred)
    
    pf = FP / (FP + TN)
    
    GMean = np.sqrt(recall * (1-pf))
    
    Balance = 1 - (np.sqrt(pf**2 + (1 - recall)**   2) / np.sqrt(2))
    
    return GMean, Balance

In [200]:
GMean_and_Balance(y_target, y_target_pred)

(0.7044384997618941, 0.6779884727261365)

In [201]:
from sklearn.metrics import precision_score

print(precision_score(y_target, y_target_pred))
print(recall_score(y_target, y_target_pred))

0.33076923076923076
0.5584415584415584


In [202]:
from sklearn.metrics import auc, roc_curve
fpr, tpr, thresholds = roc_curve(y_target, y_target_pred, pos_label=2)
auc(fpr, tpr)

c:\Users\parve\AppData\Local\Programs\Python\Python310\lib\site-packages\sklearn\metrics\_ranking.py:999: UndefinedMetricWarning: No positive samples in y_true, true positive value should be meaningless
  warnings.warn(


nan